<a href="https://colab.research.google.com/github/rayxuan2000/Project/blob/main/fraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0: Summary of Fraud Detection Code Lab
- Data is highly imbalanced
- Features of interval_after_signup and time related raw and aggregates are highly predictive of fraud
- Made actionable operation recommendations/proposal for business

# Part 1: Import Data

In [12]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import GridSearchCV


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import warnings
warnings.filterwarnings('ignore')

In [13]:
# !git clone https://github.com/loganlaioffer/fraudDetection.git

In [14]:
# ipURL = 'https://github.com/loganlaioffer/fraudDetection/blob/master/IpAddress_to_Country.csv'
# fraudURL = 'https://github.com/loganlaioffer/fraudDetection/blob/master/imbalancedFraudDF.csv'


# ipURL = 'https://drive.google.com/file/d/1d8zzSHM0VMDDzZm0QfUSUpqdpgCQHrCa/view?usp=drive_link'
# fraudURL = 'https://drive.google.com/file/d/1rw6xYravTptbdK0RskF5qwaFq7Um_KiP/view?usp=drive_link'

id1 = "1d8zzSHM0VMDDzZm0QfUSUpqdpgCQHrCa"
id2 = "1rw6xYravTptbdK0RskF5qwaFq7Um_KiP"
file1 = drive.CreateFile({'id':id1})
file1.GetContentFile('IpAddress_to_Country.csv')
file2 = drive.CreateFile({'id':id2})
file2.GetContentFile('imbalancedFraudDF.csv')

ipToCountry = pd.read_csv('IpAddress_to_Country.csv')
fraud_data = pd.read_csv('imbalancedFraudDF.csv')

# ipToCountry = pd.read_csv('fraudDetection/IpAddress_to_Country.csv')
# fraud_data = pd.read_csv('fraudDetection/imbalancedFraudDF.csv')

In [15]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0


In [16]:
ipToCountry.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


# Part 2: Data exploration

In [17]:
#Distribution of the label column
fraud_data['class'].value_counts()

0    136961
1      1415
Name: class, dtype: int64

Missing values, outlier?

In [18]:
# count of NaN in each column
fraud_data.isnull().sum()

#HW: check count of fraud that is outliers

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

In [19]:
ipToCountry['country'].value_counts()

United States                            46868
Canada                                    6989
Russian Federation                        6739
Australia                                 6316
Germany                                   5999
                                         ...  
Timor-leste                                  1
Korea Democratic People's Republic of        1
St. Pierre and Miquelon                      1
Tokelau                                      1
Saint Lucia                                  1
Name: country, Length: 235, dtype: int64

#Step 1: Identify country info based on ip_address


In [20]:
ipToCountry.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [21]:
# fraud_data = fraud_data.iloc[100000:,:]
fraud_data = fraud_data.iloc[:,:]
fraud_data.reset_index(drop=True, inplace=True)
fraud_data.head()


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0


In [22]:
start = time.time()

countries = []
for i in range(len(fraud_data)):
    ip_address = fraud_data.loc[i, 'ip_address']#number
    #check which interval does ip_address falls into
    #below [] is list of T/F, only when this ip_address falls into the correct internal row does the index generate a True
    #tmp is a df of shape n * 3, where n is 1 if found a match (ip_address falls in range) or 0 if no match
    tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
    if len(tmp) == 1:#found match
        countries.append(tmp['country'].values[0])
    else:#no match
        countries.append('NA')

# fraud_data['country'] = countries
runtime = time.time() - start

print("Ordinary lookup took", runtime, "seconds.")



Ordinary lookup took 122.47372817993164 seconds.


In [23]:
ip_address = fraud_data.loc[10, 'ip_address']
tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
print(tmp)

Empty DataFrame
Columns: [lower_bound_ip_address, upper_bound_ip_address, country]
Index: []


Optimize the above look-up process by binary search.

In [24]:
start = time.time()

countries = []

for i in range(len(fraud_data)):
    left, right = 0, len(ipToCountry)-1
    flag = 0
    while left <= right:
        mid = (left + right)//2
        if fraud_data.loc[i, 'ip_address'] >= ipToCountry.loc[mid, 'lower_bound_ip_address'] and \
            fraud_data.loc[i, 'ip_address'] <= ipToCountry.loc[mid, 'upper_bound_ip_address']:
            countries.append(ipToCountry.loc[mid, 'country'])
            flag = 1
            break
        elif fraud_data.loc[i, 'ip_address'] > ipToCountry.loc[mid, 'upper_bound_ip_address']:
            left = mid + 1
        else:
            right = mid - 1
    if flag == 0:
      countries.append('NA')

fraud_data['country'] = countries
runtime = time.time() - start

print("Binary search lookup took", runtime, "seconds.")

Binary search lookup took 95.41291213035583 seconds.


In [ ]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,266584,2015-03-25 23:06:38,2015-03-29 03:09:43,43,ZCBMXTADTIRKO,Ads,IE,M,40,7.011094e+08,0,Ghana
1,252209,2015-07-19 02:02:00,2015-09-30 13:10:33,30,YYTZLYZNLENLU,Direct,Chrome,F,37,3.739036e+09,0,China
2,55611,2015-06-26 04:06:36,2015-09-10 23:54:53,33,MIWEGSPWKLIIH,Ads,FireFox,F,29,2.014574e+09,0,Australia
3,147968,2015-05-12 01:50:28,2015-08-25 12:47:06,47,ZLELLLBYCBXMJ,Direct,Chrome,M,43,2.492607e+08,0,China
4,12072,2015-03-05 17:21:58,2015-03-22 22:00:21,43,TTUOKPOSBZXBQ,SEO,FireFox,M,42,1.479019e+09,0,Italy


Test if column user_id is unique for time related aggregates.

In [ ]:
print(fraud_data.user_id.nunique())#138376
print(len(fraud_data.index))#138376
#all of the user_id has only the first 1 transaction, no way to do time based aggregates,
#e.g. amount/counts in past 1 day for this user

38376
38376


# Part 3a: Feature Engineering




In [ ]:
#time related features: can be done before split, as they has no interaction between other rows, solely based on other columns of the same row
fraud_data['interval_after_signup'] = (pd.to_datetime(fraud_data['purchase_time']) - pd.to_datetime(
        fraud_data['signup_time'])).dt.total_seconds()

fraud_data['signup_days_of_year'] = pd.DatetimeIndex(fraud_data['signup_time']).dayofyear

#bed time operation
fraud_data['signup_seconds_of_day'] = pd.DatetimeIndex(fraud_data['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['signup_time']).hour

fraud_data['purchase_days_of_year'] = pd.DatetimeIndex(fraud_data['purchase_time']).dayofyear
fraud_data['purchase_seconds_of_day'] = pd.DatetimeIndex(fraud_data['purchase_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['purchase_time']).hour

fraud_data = fraud_data.drop(['user_id','signup_time','purchase_time'], axis=1)

In [ ]:
fraud_data.head()
#note there are NAs in country

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
0,43,ZCBMXTADTIRKO,Ads,IE,M,40,7.011094e+08,0,Ghana,273785.0,84,83198,88,11383
1,30,YYTZLYZNLENLU,Direct,Chrome,F,37,3.739036e+09,0,China,6347313.0,200,7320,273,47433
2,33,MIWEGSPWKLIIH,Ads,FireFox,F,29,2.014574e+09,0,Australia,6637697.0,177,14796,253,86093
3,47,ZLELLLBYCBXMJ,Direct,Chrome,M,43,2.492607e+08,0,China,9111398.0,132,6628,237,46026
4,43,TTUOKPOSBZXBQ,SEO,FireFox,M,42,1.479019e+09,0,Italy,1485503.0,64,62518,81,79221


In [ ]:
print(fraud_data.source.value_counts())

SEO       15431
Ads       15297
Direct     7648
Name: source, dtype: int64


# Part 4: Data Split

In [ ]:
y = fraud_data['class']
X = fraud_data.drop(['class'], axis=1)

#split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

X_train.shape: (30700, 13)
y_train.shape: (30700,)


In [ ]:
X_train['country'].value_counts(ascending=True)
#drawback: collision in the same bucket(no differentiation for these countries)

Bermuda                  1
Brunei Darussalam        1
Belize                   1
Jamaica                  1
Faroe Islands            1
                     ...  
Korea Republic of      878
Japan                 1478
China                 2421
NA                    4402
United States        11793
Name: country, Length: 148, dtype: int64

In [ ]:
X_train.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
16709,42,GUFMTYPZGHJMJ,Ads,Chrome,M,39,2.218733e+09,United States,7033341.0,94,11660,175,46601
32678,51,GHPSBKSPCKHXU,Ads,Safari,F,51,6.675351e+08,Indonesia,7020074.0,135,50812,216,72486
18035,42,YYSOGVODGEURD,Direct,Chrome,F,38,1.960641e+09,Japan,6561057.0,220,77522,296,72179
14255,30,XGVSONWQQKXFW,SEO,FireFox,M,32,4.193853e+09,NA,3653816.0,202,23811,244,48827
24492,31,OSDVSSBHTRRRR,SEO,Safari,F,28,4.158374e+09,NA,9723312.0,17,73299,130,33411


# Part 3b: Feature Engineering


Convert categorical features with high cadinality to numericals

In [ ]:
#converting needs to be done after split
X_train = pd.get_dummies(X_train, columns=['source', 'browser'])#need to drop ['source', 'browser']? no, auto dropped by get_dummies
X_train['sex'] = (X_train.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
# if device_id abc occurred 100 times in X_train, then replace all abc in device_id col in X_train by 100
X_train_device_id_mapping = X_train.device_id.value_counts(dropna=False)
X_train['n_dev_shared'] = X_train.device_id.map(X_train_device_id_mapping)# number of times device_id occurred in train data

# the more a ip is shared, the more suspicious
X_train_ip_address_mapping = X_train.ip_address.value_counts(dropna=False)
X_train['n_ip_shared'] = X_train.ip_address.map(X_train_ip_address_mapping)

# the less visit from a country, the more suspicious
X_train_country_mapping = X_train.country.value_counts(dropna=False)#include counts of NaN
X_train['n_country_shared'] = X_train.country.map(X_train_country_mapping)#lots of NAs in country column, #without dropna=False will produce nan in this col


X_train = X_train.drop(['device_id','ip_address','country'], axis=1)




In [ ]:
##########################
X_test = pd.get_dummies(X_test, columns=['source', 'browser'])
X_test['sex'] = (X_test.sex == 'M').astype(int)


# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html
# the more a device is shared, the more suspicious
# X_test['n_dev_shared'] = X_test.device_id.map(X_train_device_id_mapping)

# the more a ip is shared, the more suspicious
# X_test['n_ip_shared'] = X_test.ip_address.map(X_train_ip_address_mapping)

# the less visit from a country, the more suspicious
# X_test['n_country_shared'] = X_test.country.map(X_train_country_mapping)

#but here device_id, ip_address has few overlap between train and test, if we apply the above
# 3 X_train_mappings (generated from X_train) on X_test, then most of the X_test will be NaN, as we can not find the keys in the train mapping




# if apply train mapping, most of the levels in test does not occur in train, so most are null after converting, so redo mapping on test data, which we should not

# the more a device is shared, the more suspicious
X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)

# if the levels/values/mapping keys of the column in train and test data are pretty much the same(lots of overlap), e.g. country,
# then we should apply the above 3 X_train_mappings (generated from X_train) on X_test(like below),
# rather than using the new mapping generated from X_test



In [ ]:
X_train.head()

,purchase_value,sex,age,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,n_dev_shared,n_ip_shared,n_country_shared
16709,42,1,39,7033341.0,94,11660,175,46601,1,0,0,1,0,0,0,0,1,1,11793
32678,51,0,51,7020074.0,135,50812,216,72486,1,0,0,0,0,0,0,1,1,1,130
18035,42,0,38,6561057.0,220,77522,296,72179,0,1,0,1,0,0,0,0,1,1,1478
14255,30,1,32,3653816.0,202,23811,244,48827,0,0,1,0,1,0,0,0,1,1,4402
24492,31,0,28,9723312.0,17,73299,130,33411,0,0,1,0,0,0,0,1,1,1,4402


Use the same scaler for both train and test data.

In [ ]:
# # normalize (min-max) to [0,1], standardize(StandardScaler) to normal, mu=0,var = 1 can < 0, so we do normalize here

# needs to be brought to the same scale for models like LR with regularization(that are not tree based)

#Compute the train minimum and maximum to be used for later scaling:
scaler = preprocessing.MinMaxScaler().fit(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])
#print(scaler.data_max_)

#transform the training data and use them for the model training
X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = scaler.transform(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])

#before the prediction of the test data, apply the same scaler obtained from above, on X_test, not fitting a brandnew scaler on test
X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = scaler.transform(X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])



In [ ]:
X_train.n_dev_shared.value_counts(dropna=False)

0.00    29819
0.25      650
0.50      174
0.75       52
1.00        5
Name: n_dev_shared, dtype: int64

In [ ]:
X_test.n_dev_shared.value_counts(dropna=False)

0.00    7624
0.25      42
0.50       6
0.75       4
Name: n_dev_shared, dtype: int64

# Part 5: Model Training


In [ ]:
fraud_data['class'].value_counts()

0    36961
1     1415
Name: class, dtype: int64

### Simple LogisticRegression model

In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

# predict on test
y_pred=logreg.predict(X_test)

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))

        pred_0  pred_1
true_0    7390       0
true_1     286       0
recall = 0.0


We can see from the above result that the model just predicts every fraud case as normal. That's due to the imbalanced distribution between positive and negative samples. So later we need to consider this problem.

### Simple RF model

In [ ]:
classifier_RF = RandomForestClassifier(random_state=0)

classifier_RF.fit(X_train, y_train)

# generate class raw probabilities
probs = classifier_RF.predict_proba(X_test)

# predict class labels 0/1 for the test set
predicted = classifier_RF.predict(X_test)

# # generate evaluation metrics
# print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
# print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
# print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
# print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0predicted = classifier_RF.predict(X_test)

confusion_matrix is: 
        pred_0  pred_1
true_0    7389       1
true_1     148     138
recall = 0.4825174825174825


The RF result is relatively better than LR. But it is still worth improving.

### SMOTE sampling

Here we just synthesizes new minority instances by interpolating between exisiting ones.

In [ ]:
#Wrong way to sampling: smote before split train/test, then test data does not reflect true distribution in reality,
#and “blend” information from the test set into the training of the model. overfit! think about the case of simple
#oversampling (where I just duplicate observations). If I upsample a dataset before splitting it into a train and
#validation set, I could end up with the same observation in both datasets

#https://imbalanced-learn.org/en/stable/install.html

# Install
# imbalanced-learn is currently available on the PyPi’s reporitories and you can install it via pip:

# pip install -U imbalanced-learn

#oversampling on only the training data, the right way!
#sampling_strategy = number of samples in the majority class over the number of samples in the minority class after resampling
# k = [3,5,7,9,11]
# res = 0
# opt_k = k[0]
# for i in range(len(k)):
#     smote = SMOTE(random_state=12, k_neighbors=k[i])
#     x_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
#     classifier_RF_sm = RandomForestClassifier(random_state=0)
#     classifier_RF_sm.fit(x_train_sm, y_train_sm)
#     predicted_sm = classifier_RF_sm.predict(X_test)
#     if f1_score(y_test, predicted_sm) > res:
#       opt_k = k[i]
#     res = max(res, f1_score(y_test, predicted_sm))
# print("Best F1-score is:", res)
# print("Best parameter is:", opt_k)
###########################################################
smote = SMOTE(random_state=3, k_neighbors=3)
x_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

unique, counts = np.unique(y_train_sm, return_counts=True)

print(np.asarray((unique, counts)).T)

[[    0 29571]
 [    1 29571]]


We can see from above that the distribution is equally now.

In [ ]:
#RF on smoted training data
classifier_RF_sm = RandomForestClassifier(random_state=0)

classifier_RF_sm.fit(x_train_sm, y_train_sm)

# predict class labels for the test set
predicted_sm = classifier_RF_sm.predict(X_test)

# generate class probabilities
probs_sm = classifier_RF_sm.predict_proba(X_test)


# # generate evaluation metrics
# print("%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
# print("%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
# print("%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm )))#string to int

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall or sens_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1]))
# print('precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1]))

confusion_matrix_sm is: 
        pred_0  pred_1
true_0    7388       2
true_1     148     138
recall or sens_sm = 0.4825174825174825


The result is almost as same as the original RF one. It's time to use parameter tuning.

# Part 6: Parameter tuning by GridSearchCV

Eval metrics for GridSearchCV over all fits upon combination of parameters and cv

In [ ]:
# Eval metrics to be calculated for each combination of parameters, they are all calculated(by scoring), but not necessarily use all of them to pick the optimal(only use refit_score)
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score, pos_label=1)

}

In [ ]:
def grid_search_wrapper(model, parameters, refit_score='recall_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """

    grid_search = GridSearchCV(model, parameters, scoring=scorers, refit=refit_score,
                           cv=3, return_train_score=True)
    grid_search.fit(X_train, y_train)

    # make the predictions
    y_pred = grid_search.predict(X_test)
    y_prob = grid_search.predict_proba(X_test)[:, 1]

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # # confusion matrix on the test data.
    # print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    # cm = confusion_matrix(y_test, y_pred)
    # cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
    # print(cmDF)

    # print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("recall_score is: ", f1_score(y_test, y_pred)))#string to int

    # print('recall = ', float(cm[1,1]) / (cm[1,0] + cm[1,1]))
    # print('precision = ', float(cm[1,1]) / (cm[1, 1] + cm[0,1]))

    return grid_search



### Optimizing on recall_score on LR

In [ ]:
# C: inverse of regularization strength, smaller values specify stronger regularization
# LRGrid = {"C" : np.logspace(-2,2,5), "penalty":["l1","l2"]}# l1 lasso l2 ridge
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
logRegModel = LogisticRegression(random_state=0)

grid_search_LR_recall = grid_search_wrapper(logRegModel, param_grid, refit_score='recall_score')

Best params for recall_score
{'C': 0.01, 'penalty': 'l2'}
	recall_score is: : 0.0


### Optimizing on recall_score on RF

In [ ]:
parameters = {
#None: nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples
'max_depth': [10,11,12,13],
'n_estimators' :  [8,9,10,11],
'class_weight' : [{0: 1, 1: w} for w in [5,6,7,8,9]]
}

clf = RandomForestClassifier(random_state=0)

In [ ]:
grid_search_rf_recall = grid_search_wrapper(clf, parameters, refit_score='recall_score')#no improvement on f1

Best params for recall_score
{'class_weight': {0: 1, 1: 8}, 'max_depth': 13, 'n_estimators': 10}
	recall_score is: : 0.6509433962264151


In [ ]:
best_rf_model_recall = grid_search_rf_recall.best_estimator_
best_rf_model_recall

RandomForestClassifier(class_weight={0: 1, 1: 8}, max_depth=13, n_estimators=10,
                       random_state=0)

In [ ]:
# predict class labels for the test set
predictedBest_recall = best_rf_model_recall.predict(X_test)

# generate class probabilities
probsBest_recall = best_rf_model_recall.predict_proba(X_test)

results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)# recall score is different from above, as above is metric on test data, this is performance on cv data
results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()
#recall is worse than default rf?? no this is on test, but train recall is better

,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
62,0.997,0.534,0.695,1.000,0.610,0.758,13,"{0: 1, 1: 8}",10
78,0.992,0.533,0.693,0.999,0.609,0.757,13,"{0: 1, 1: 9}",10
77,0.991,0.533,0.693,0.999,0.611,0.758,13,"{0: 1, 1: 9}",9
34,0.993,0.533,0.694,0.999,0.568,0.724,10,"{0: 1, 1: 7}",10
47,0.995,0.533,0.694,1.000,0.600,0.750,13,"{0: 1, 1: 7}",11


In [ ]:
results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)
results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()


,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
62,0.997,0.534,0.695,1.000,0.610,0.758,13,"{0: 1, 1: 8}",10
78,0.992,0.533,0.693,0.999,0.609,0.757,13,"{0: 1, 1: 9}",10
77,0.991,0.533,0.693,0.999,0.611,0.758,13,"{0: 1, 1: 9}",9
34,0.993,0.533,0.694,0.999,0.568,0.724,10,"{0: 1, 1: 7}",10
47,0.995,0.533,0.694,1.000,0.600,0.750,13,"{0: 1, 1: 7}",11


In [ ]:
#Var Importance, make a guess?
pd.DataFrame(best_rf_model_recall.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)




,importance
purchase_days_of_year,0.332575
interval_after_signup,0.322574
n_dev_shared,0.066826
n_ip_shared,0.063310
purchase_seconds_of_day,0.041600
signup_days_of_year,0.036123
signup_seconds_of_day,0.033077
purchase_value,0.030280
age,0.025950
n_country_shared,0.019139


### Optimizing recall_score on RF

In [ ]:
# grid_search_rf_recall = grid_search_wrapper(clf, parameters, refit_score='recall_score')

In [ ]:
# best_RF_model_recall = grid_search_rf_recall.best_estimator_
# best_RF_model_recall

In [ ]:
# # predict class labels for the test set
# predictedBest_recall = best_RF_model_recall.predict(X_test)

# # generate class probabilities
# probsBest_recall = best_RF_model_recall.predict_proba(X_test)

# results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)# recall score is different from above, as above is metric on test data, this is performance on cv data
# results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
# results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()
# #recall is worse than default rf?? no this is on test, but train recall is better

# Tasks 3: Fraud Characteristics

In [ ]:
#for task 3, based on the above var importance
trainDF = pd.concat([X_train, y_train], axis=1)
pd.crosstab(trainDF["n_dev_shared"],trainDF["class"])
#the larger n_dev_shared, the higher rate of fraud

class,0,1
n_dev_shared,,
0.00,29143,676
0.25,381,269
0.50,38,136
0.75,9,43
1.00,0,5


In [ ]:
fraud_data.groupby("class")[['interval_after_signup']].mean()#action velocity(consecutive operations/actions of user)
#interval_after_signup on frauds are significantly lower compared to legits

,interval_after_signup
class,
0,5.194299e+06
1,2.570226e+06


In [ ]:
fraud_data.groupby("class")[['interval_after_signup']].median()#1
#more than half of fraud happened 1s after signed up

,interval_after_signup
class,
0,5188824.0
1,1.0


In [ ]:
fraud_data[fraud_data['class'] == 1].head(100)

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
36961,24,VLHGCDPFCICDA,SEO,Chrome,F,33,3.432126e+09,1,United States,3327952.0,218,80113,257,38465
36962,14,YLUQSRNYYIPXU,Ads,Chrome,M,40,3.905319e+09,1,NA,1.0,12,4207,12,4208
36963,63,ABUBCQDATQMQH,Ads,FireFox,F,46,5.505670e+08,1,United States,7640070.0,49,40723,137,77593
36964,34,QHEODGCAVJKIQ,SEO,Chrome,M,37,9.408096e+08,1,United States,1.0,12,77710,12,77711
36965,76,DAKVYHKIEYRBH,SEO,Chrome,F,48,6.361041e+08,1,Hungary,1.0,10,48421,10,48422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37056,21,AIGPGDVRDKOKT,SEO,FireFox,F,45,3.058136e+09,1,India,1.0,2,69762,2,69763
37057,52,PQGKGQACIARBV,Ads,Chrome,F,36,7.203338e+08,1,China,1.0,9,77018,9,77019
37058,54,XHZBVWFWHSGTQ,SEO,FireFox,M,50,2.249217e+09,1,United States,1.0,1,32483,1,32484
37059,10,WETYPHOQVLWMK,Ads,FireFox,M,36,2.293333e+09,1,United States,1.0,7,63914,7,63915


#Task 4: How to use the prediction

In [ ]:
results_sortrecall['mean_test_recall_score']

62    0.534107
78    0.533220
77    0.533220
34    0.533220
47    0.533220
        ...   
53    0.531447
18    0.531447
44    0.530560
52    0.530560
32    0.530560
Name: mean_test_recall_score, Length: 80, dtype: float64

In [ ]:
t = (10 * probsBest_recall[:, 1]).astype(int)
unique, counts = np.unique(t, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 5679]
 [   1 1767]
 [   2   83]
 [   3    7]
 [   4    1]
 [   5    1]
 [   6    1]
 [   7    7]
 [   8   16]
 [   9   51]
 [  10   63]]


The above result states the probability of a user to have a fraud activity. The company can divide the whole group into three parts,
0\~4 represent normal users, 5\~7 means further investigation is needed, 8\~10 can be directly rejected.